In [2]:
import csv
import glob
import os
import random

from datasets import load_dataset, Dataset, DatasetDict

In [3]:
dataset = load_dataset("bitext/customer-support-intent-dataset")

Extracting data files:   0%|          | 0/3 [00:00<?, ?it/s]

Generating train split: 0 examples [00:00, ? examples/s]

Generating validation split: 0 examples [00:00, ? examples/s]

Generating test split: 0 examples [00:00, ? examples/s]

Dataset csv downloaded and prepared to /home/diwank/.cache/huggingface/datasets/bitext___csv/bitext--customer-support-intent-dataset-faed7337b63b4f3a/0.0.0/6954658bab30a358235fa864b05cf819af0e179325c740e4bc853bcc7ec513e1. Subsequent calls will reuse this data.


  0%|          | 0/3 [00:00<?, ?it/s]

In [11]:
def not_in(tags, remove_tags):
    return all([
        remove_tag not in tags
        for remove_tag in remove_tags
    ])

# See: https://huggingface.co/datasets/bitext/customer-support-intent-dataset
filtered = dataset.filter(lambda row: row["category"] != "NEWSLETTER" and not_in(row["tags"], list("ZWKMLEQ")))
len(filtered["train"]), len(dataset["train"])

Filter:   0%|          | 0/6539 [00:00<?, ? examples/s]

Filter:   0%|          | 0/818 [00:00<?, ? examples/s]

Filter:   0%|          | 0/818 [00:00<?, ? examples/s]

(3267, 6539)

In [14]:
# The categories and intents covered by the dataset are:

intents_by_category = dict(
    ACCOUNT="create_account, delete_account, edit_account, recover_password, registration_problems, switch_account".split(", "),
    CANCELLATION_FEE="check_cancellation_fee".split(", "),
    CONTACT="contact_customer_service, contact_human_agent".split(", "),
    DELIVERY="delivery_options, delivery_period".split(", "),
    FEEDBACK="complaint, review".split(", "),
    INVOICE="check_invoice, get_invoice".split(", "),
    # NEWSLETTER="newsletter_subscription".split(", "),
    ORDER="cancel_order, change_order, place_order, track_order".split(", "),
    PAYMENT="check_payment_methods, payment_issue".split(", "),
    REFUND="check_refund_policy, get_refund, track_refund".split(", "),
    SHIPPING_ADDRESS="change_shipping_address, set_up_shipping_address".split(", "),
)

In [13]:
dataset["train"][0]

{'utterance': 'would it be possible to cancel the order I made?',
 'intent': 'cancel_order',
 'category': 'ORDER',
 'tags': 'BIP'}

In [17]:
make_chatml = lambda name, role, content: dict(
    name=name, role=role, content=content,
)

system = lambda name, content: make_chatml(
    role="system",
    name=name,
    content=content,
)

situation = lambda content: system(name="situation", content=content)
thought = lambda content: system(name="thought", content=content)
information = lambda content: system(name="information", content=content)
me = lambda content, name=None: make_chatml(
    role="assistant",
    content=content,
    name=name,
)

person = lambda content, name=None: make_chatml(
    role="user",
    content=content,
    name=name,
)

def to_chatml(row):
    utterance = row["utterance"]
    intent = row["intent"]
    category = row["category"]

    allowed_intents = intents_by_category[category]

    choice_keys = list("ABCDEFG")
    choice = choice_keys[allowed_intents.index(intent)]
    allowed_intents_text = [
        allowed_intent.replace("_", " ").capitalize()
        for allowed_intent in allowed_intents
    ]
    
    intent_text = intent.replace("_", " ").capitalize()

    chatml = [
        situation(
            f"AI Assistant is talking to customer to provide support."
        ),
        person(
            utterance.capitalize(),
            name="Customer",
        ),
        thought(
            f"Which of the following best describe the customer's intent?"
            + "\nChoices:\n"
            + '\n'.join([
                f"{key}) {allowed_intent}"
                for key, allowed_intent in zip(choice_keys, allowed_intents_text)
            ])
            + '\n'
            + f"\nAnswer: {choice} ({intent_text})"
        ),
    ]

    return dict(chatml=chatml)

In [19]:
filtered = filtered.map(to_chatml).remove_columns(list(set(dataset["train"].column_names) - {"chatml"}))

Map:   0%|          | 0/3267 [00:00<?, ? examples/s]

Map:   0%|          | 0/409 [00:00<?, ? examples/s]

Map:   0%|          | 0/427 [00:00<?, ? examples/s]

In [20]:
filtered["train"][0]

{'chatml': [{'content': 'AI Assistant is talking to customer to provide support.',
   'name': 'situation',
   'role': 'system'},
  {'content': 'Would it be possible to cancel the order i made?',
   'name': 'Customer',
   'role': 'user'},
  {'content': "Which of the following best describe the customer's intent?\nChoices:\nA) Cancel order\nB) Change order\nC) Place order\nD) Track order\n\nAnswer: A (Cancel order)",
   'name': 'thought',
   'role': 'system'}]}

In [21]:
filtered.push_to_hub("diwank/samantha-intent", private=True)

Pushing split train to the Hub.


Pushing dataset shards to the dataset hub:   0%|          | 0/1 [00:00<?, ?it/s]

Creating parquet from Arrow format:   0%|          | 0/4 [00:00<?, ?ba/s]

Upload 1 LFS files:   0%|          | 0/1 [00:00<?, ?it/s]

Pushing split validation to the Hub.


...

Upload 1 LFS files:   0%|          | 0/1 [00:00<?, ?it/s]

Pushing split test to the Hub.


Pushing dataset shards to the dataset hub:   0%|          | 0/1 [00:00<?, ?it/s]

Creating parquet from Arrow format:   0%|          | 0/1 [00:00<?, ?ba/s]

Upload 1 LFS files:   0%|          | 0/1 [00:00<?, ?it/s]